1. Use the “Classic Outlook” (needed for win32com)
    win32 works with classic outlook and it needs microsoft 365 subscription
    
2. If you stick with Microsoft Store Outlook
   You cannot use win32com — because it has no COM interface.

In [1]:
# pip install pywin32

In [1]:
# outlook.bat is used for adding outlook.exe to system path, this is not used

### import Libs

In [35]:
import win32com.client as win32
import os
import time

### Variables

In [13]:
MailBoxName = '@gmail.com'
ToRecipients = '@gmail.com'
ReplyRecipients = '@gmail.com'
AttachmentFilePath = '"C:\Users\UserName\OneDrive\Desktop\My Work Experience\intelligent-automation-python\README.md"'

### Connect to Outlook

In [5]:
## for checking 64 bit or 32 bit 
# import platform
# print(platform.architecture())

In [7]:
# os.startfile('outlook')

In [26]:
outlook = win32.Dispatch("Outlook.Application")

### Send Email through Outlook

In [27]:
name_space = outlook.GetNamespace("MAPI") 

In [28]:
# selecting the profile
profile = name_space.Folders[MailBoxName]

In [29]:
# selecting the Sender Account
SenderAccount = outlook.Session.Accounts[MailBoxName]

In [32]:
email = outlook.CreateItem(0)

In [34]:
body_html = '''<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>Message</title>
  <style>
    body { font-family: Arial, Helvetica, sans-serif; font-size: 14px; color: #111; }
    .container { max-width: 680px; margin: 20px auto; padding: 16px; }
  </style>
</head>
<body>
  <div class="container">
    <p>Hi,</p>

    <p>PFA,</p>

    <p>Thanks and Regards,</p>
  </div>
</body>
</html>
'''

In [33]:
email.To = ToRecipients
email.ReplyRecipients.Add(ReplyRecipients)
email.Subject = 'Test Subject'
email.HTMLBody = body_html   # HTML Format
# email.Body = MailBody        # normal Format
email.Attachments.Add(AttachmentFilePath)
email.Save()
time.sleep(2)

In [ ]:
email.Send()